In [2]:
import pandas as pd
import numpy as np


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from bs4 import BeautifulSoup

import string
import re


from lightgbm import LGBMClassifier
from sklearn.feature_extraction.text import HashingVectorizer, TfidfVectorizer, CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score



C:\Users\Jacob\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [8]:
# Start selenium driver
driver = webdriver.Chrome(executable_path='./chromedriver/chromedriver.exe')
# jobs to search for
jobs = ['data-scientist', 'data-analyst', 'business-analyst','research-scientist','machine-learning-engineer']
#init list for raw-html
raw = []
for job in jobs:
    # Search through each job
    url = 'https://www.seek.com.au/'+job+'-jobs/'
    
    driver.get(url)
    WebDriverWait(driver, 3)

    pages = 12
    
    
    for page in range(2,pages):
        #iterate through each page of job keyword search and ad each link of job ad to a list
        ads = driver.find_elements_by_css_selector('._2iNL7wI')
        links = [ad.get_attribute('href') for ad in ads]
        #iterate through each link but skip the first 2 which are featured ads that repeat themselves each page
        for link in links[2:]:
            # go to link address
            driver.get(link)
            WebDriverWait(driver, 3)
            # get title and raw html of job ad and add to raw list
            title = driver.find_element_by_css_selector('h1').text
            raw_html = driver.page_source
            raw.append([title, raw_html])
            driver.back()
            WebDriverWait(driver, 3)
        # go to next page
        driver.get(url+'?page='+str(page))
# save raw to a Dataframe and save
raw_df = pd.DataFrame(raw,columns=['job_title','raw_html'])
driver.close()
raw_df.to_csv('./raw_data.csv', index=False)

In [9]:
raw_df = pd.read_csv('./raw_data.csv')

In [10]:
raw_df.head()

,job_title,raw_html
0,Data Scientist | Machine Learning & AI,"<html lang=""en"" prefix=""og: http://ogp.me/ns#""..."
1,Principal Data Scientist,"<html lang=""en"" prefix=""og: http://ogp.me/ns#""..."
2,Data Scientist,"<html lang=""en"" prefix=""og: http://ogp.me/ns#""..."
3,Data Scientist,"<html lang=""en"" prefix=""og: http://ogp.me/ns#""..."
4,Senior Data Scientist,"<html lang=""en"" prefix=""og: http://ogp.me/ns#""..."


In [ ]:
# checking what parts of raw_html to extract and assign to a column

# raw_df.raw_html = raw_df.raw_html.apply(lambda x: BeautifulSoup(x).text)
# soup = BeautifulSoup(raw_df.raw_html[1])
# Advertiser_name = soup.find(attrs={'data-automation':'advertiser-name'}).text
# Job_Description = soup.find(attrs={'data-automation':'mobileTemplate'}).text
# Date = soup.find(attrs={'data-automation':'job-detail-date'}).text
# soup.findChildren(attrs={'class':'lwHBT6d'})


In [ ]:
raw_df.job_description[raw_df.job_description.isna()]

In [ ]:
raw_df.iloc[435,:]

In [ ]:
raw_df.job_title.unique()

In [ ]:
raw_df.drop([435],inplace=True)

In [ ]:
raw_df['job_description'] = raw_df.raw_html.apply(lambda x: BeautifulSoup(x).find(attrs={'data-automation':'mobileTemplate'}).text)

In [ ]:
raw_df['job_info'] = raw_df.raw_html.apply(lambda x: BeautifulSoup(x).findChildren(attrs={'class':'lwHBT6d'}))

In [1]:
#raw_df is very messy so these are some functions to help clean it up
def text_maker(x):
    lis = []
    for i in x:
        lis.append(i.text)
    return lis

def cleaner(elem):
    elem = elem.split(',')
    del elem[0]
    return elem

In [ ]:
raw_df['job_info_text'] = raw_df.job_info.apply(text_maker)

In [ ]:
raw_df.head()

In [ ]:
draft_df = raw_df.drop(['raw_html','job_info'],axis=1)

In [ ]:
draft_df.head()

In [ ]:
draft_df.to_csv('./draft_df.csv',index=False)

In [ ]:
draft_df = pd.read_csv('./draft_df.csv')

In [ ]:
draft_df.job_info_text = draft_df.job_info_text.apply(lambda x: cleaner(x))

In [ ]:
draft_df.head()

In [ ]:
draft_df.job_info_text[0][0]

In [ ]:
draft_df['title'] = draft_df.job_info_text.apply(lambda x: x.pop(0))

In [ ]:
draft_df['data_listed'] = draft_df.job_info_text.apply(lambda x: x.pop(0))

In [ ]:
draft_df['location'] = draft_df.job_info_text.apply(lambda x: x.pop(0))

In [ ]:
draft_df.tail()

In [ ]:
draft_df.drop(['title'],inplace=True,axis=1)

In [ ]:
draft_df['salary'] = draft_df.job_info_text.apply(lambda x: x.pop(0))

In [ ]:
draft_df.salary = draft_df.salary.apply(lambda x: x.strip("' '"))

In [ ]:
def findsalary(x):
    salary = re.findall(pattern=r'\$(\d*\w)',string=x)
    try:
        return salary
    except:
        return np.nan

In [ ]:
findsalary(draft_df.salary[0])

In [ ]:
draft_df.salary = draft_df.salary.apply(findsalary)

In [ ]:
draft_df.head()

In [ ]:
draft_df.salary

In [ ]:
# def crapsorter(x):
#     if x == [None]:
#         x = np.nan
#     elif len(x)==1:
#         try:
#             x = int(x)
#         except:
#             x = str(x)

In [ ]:
#draft_df.salary = draft_df.salary.apply(crapsorter)

In [ ]:
draft_df.tail()

In [ ]:
draft_df = draft_df[draft_df['salary'].map(lambda d: len(d)) > 0]

In [ ]:
draft_df.head()

In [ ]:
draft_df.drop(['job_info_text'],inplace=True,axis=1)

In [ ]:
draft_df.reset_index(drop=True,inplace=True)

In [ ]:
def bingbong(x):
    for i in x:
        # if salary contains k replace with 000
        i = float(i.lower().replace('k','000'))
        # if salary is between 300 and 1000: salary*365
        if (i > 300) & (i < 1000):
            i *= 365
        elif (i>100)&(i < 300):
            i *= 1000
        elif i <=100:
            i *= 42*52
        
        return i 

In [ ]:
draft_df.head()

In [ ]:
bingbong(draft_df.salary[4])

In [ ]:
draft_df.salary = draft_df.salary.apply(bingbong)

In [ ]:
draft_df.salary.unique()

In [ ]:
draft_df.salary[draft_df.salary == 0]

In [ ]:
draft_df.drop(129,inplace=True)

In [ ]:
draft_df.salary[draft_df.salary == 0]

In [ ]:
draft_df.salary[draft_df.salary == 4094]

In [ ]:
draft_df.iloc[147:150]

In [ ]:
draft_df[draft_df.duplicated(keep=False)].sort_values('job_description')

In [ ]:
df_1 = draft_df.drop_duplicates()

In [ ]:
df_1.shape

In [ ]:
df_1.to_csv('./df_1.csv',index=False)

In [ ]:
df_1 = pd.read_csv('./df_1.csv')

In [ ]:
df_1

In [ ]:
df_1.job_title.unique()

In [ ]:
def remove_numbers(x):
    x = re.findall(pattern=r'\D+\b',string=x)
    return x

In [ ]:
df_1.job_title = df_1.job_title.apply(remove_numbers)

In [ ]:
df_1.job_description = df_1.job_description.apply(remove_numbers)

In [ ]:
df_1.head()

In [ ]:
df_1.job_description = df_1.job_description.apply(lambda x: ' '.join(x))

In [ ]:
df_1.job_title = df_1.job_title.apply(lambda x: ' '.join(x))

In [ ]:
df_1.job_title

In [ ]:
df_1.to_csv('./clean_df.csv',index=False)